# AffectNet Results

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

pd.options.display.float_format = "{:,.2f}".format

In [2]:
def extract_metrics_gpu(filename): 
    f = open(filename, 'r')
    lines = f.readlines() 
    ram_init = None
    ram_peak = None
    cpu_init = None 
    cpu_end = None 
    for line in lines: 
        if "=> Average precision" in line: 
            accuracy = float(line.split(" ")[-1])
        if "=> Total training time" in line: 
            training_time = float(line.split(" ")[-2])
        if "RAM BEFORE TRAINING:" in line: 
            ram_init = float(line.split(" ")[-1])
        if "PEAK TRAINING RAM:" in line: 
            ram_peak = float(line.split(" ")[-1])
        if "CPU BEFORE TRAINING:" in line: 
            cpu_init = eval(line.split(":")[-1])
        if "CPU BEFORE EVALUATION:" in line: 
            cpu_end = eval(line.split(":")[-1])
        if "GPU BEFORE EVALUATION:" in line: 
            gpu_u, gpu_mem = eval(line.split(":")[-1])
        
    cpu_u, cpu_k = (cpu_end[0] - cpu_init[0], cpu_end[1] - cpu_init[1])
    return (accuracy * 100, training_time, (ram_peak-ram_init) * 1024, cpu_u + cpu_k, gpu_u, gpu_mem)

In [3]:
def calculate_metrics_gpu(files): 
    metrics = dict()
    
    for file in files: 
        method, seed = file.split("_") 
        
        if method in metrics: 
            metrics[method].append(extract_metrics_gpu(file))
        else: 
            metrics[method] = [extract_metrics_gpu(file)] 
    
    return metrics

In [4]:
def get_means(metrics): 
    return {method: [sum(x)/len(x) for x in list(zip(*metrics[method]))] for method in metrics}

In [5]:
def get_sds(metrics): 
    return {method: [np.std(x) for x in list(zip(*metrics[method]))] for method in metrics}

## 1. Task-IL Results (down-sampled dataset)

In [6]:
%cd ~/Desktop/research_project_repo/research_project/experiments/affectnet/2021-05-04-00-14/

/Users/samuilstoychev/Desktop/research_project_repo/research_project/experiments/affectnet/2021-05-04-00-14


In [7]:
logs = !ls

In [8]:
affectnet_metrics = calculate_metrics_gpu(logs)
affectnet_means = get_means(affectnet_metrics)
affectnet_sds = get_sds(affectnet_metrics)

In [9]:
df = pd.DataFrame(affectnet_means, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)'])
df = df.filter(["nr", "lr", "gr", "lgr", "grd", "lgrd"])

In [10]:
df

,nr,lr,gr,lgr,grd,lgrd
Accuracy (%),63.83,64.22,61.85,59.50,63.16,62.93
Training time (s),"1,473.50","1,391.87","1,869.50","1,437.47","1,858.63","1,435.30"
RAM Usage (MB),47.58,51.97,46.89,49.12,49.39,49.21
CPU Usage (s),"1,580.72","1,496.61","1,971.74","1,539.49","1,960.50","1,540.77"
GPU Usage (%),22.22,14.34,36.12,15.02,34.90,15.22
GPU Memory (MB),"2,826.00",218.67,"4,348.00",14.00,"4,348.00",14.00


## 2. Task-IL Results (fully-balanced dataset)

In [11]:
%cd ~/Desktop/research_project_repo/research_project/experiments/affectnet/2021-05-04-14-11/

/Users/samuilstoychev/Desktop/research_project_repo/research_project/experiments/affectnet/2021-05-04-14-11


In [12]:
logs = !ls

In [13]:
def split_orderings(metrics): 
    res = {}
    for m in metrics: 
        for i in range(3): 
            res[m + str(i + 1)] = metrics[m][i]
    return res

In [14]:
affectnet_metrics = calculate_metrics_gpu(logs)
affectnet_split = split_orderings(affectnet_metrics)

In [15]:
df = pd.DataFrame(affectnet_split, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)'])

In [16]:
methods = ["nr", "lr", "gr", "lgr", "grd", "lgrd"]
order = [] 
for method in methods: 
    for i in range(1, 4): 
        order.append(method + str(i))
df.filter(order)

,nr1,nr2,nr3,lr1,lr2,lr3,gr1,gr2,gr3,lgr1,lgr2,lgr3,grd1,grd2,grd3,lgrd1,lgrd2,lgrd3
Accuracy (%),67.33,65.87,62.85,67.57,67.05,62.18,63.27,62.12,57.45,63.65,60.18,59.78,67.83,66.67,61.15,66.87,67.33,64.70
Training time (s),"1,517.00","1,542.20","1,530.70","1,381.40","1,397.40","1,416.20","1,909.40","1,872.50","1,902.20","1,403.60","1,489.40","1,466.70","1,905.90","1,916.90","1,908.60","1,463.50","1,464.00","1,508.60"
RAM Usage (MB),46.62,32.07,46.89,46.24,46.52,44.61,40.26,63.75,39.96,46.28,46.16,45.18,40.23,47.99,40.30,44.52,45.56,44.28
CPU Usage (s),"1,622.33","1,645.63","1,629.65","1,482.81","1,500.15","1,519.78","2,009.60","1,971.14","2,010.24","1,506.10","1,592.40","1,566.82","2,003.82","2,019.16","2,011.21","1,566.65","1,565.93","1,608.44"
GPU Usage (%),21.78,20.88,21.75,14.77,14.42,14.54,36.51,34.96,37.57,15.81,15.24,15.75,36.37,35.70,35.17,15.39,15.79,14.52
GPU Memory (MB),"2,746.00","2,706.00","2,826.00",220.00,212.00,216.00,"4,348.00","4,348.00","4,348.00",14.00,14.00,14.00,"4,348.00","4,348.00","4,348.00",14.00,14.00,14.00


## 3. Class-IL Results

In [17]:
%cd ~/Desktop/research_project_repo/research_project/experiments/affectnet_class/2021-05-12-13-22

/Users/samuilstoychev/Desktop/research_project_repo/research_project/experiments/affectnet_class/2021-05-12-13-22


In [18]:
logs = !ls

In [19]:
affectnet_metrics = calculate_metrics_gpu(logs)
affectnet_means = get_means(affectnet_metrics)
affectnet_sds = get_sds(affectnet_metrics)

In [20]:
def split_orderings(metrics): 
    res = {}
    for m in metrics: 
        for i in range(3): 
            res[m + str(i + 1)] = metrics[m][i]
    return res

In [21]:
affectnet_metrics = calculate_metrics_gpu(logs)
affectnet_split = split_orderings(affectnet_metrics)

In [22]:
df = pd.DataFrame(affectnet_split, index=['Accuracy (%)', 
                                 'Training time (s)', 
                                 'RAM Usage (MB)', 
                                 'CPU Usage (s)', 
                                 'GPU Usage (%)', 
                                 'GPU Memory (MB)'])

In [23]:
methods = ["nr", "lr", "gr", "lgr", "grd", "lgrd"]
order = [] 
for method in methods: 
    for i in range(1, 4): 
        order.append(method + str(i))
df.filter(order)

,nr1,nr2,nr3,lr1,lr2,lr3,gr1,gr2,gr3,lgr1,lgr2,lgr3,grd1,grd2,grd3,lgrd1,lgrd2,lgrd3
Accuracy (%),24.28,22.80,24.97,25.15,23.83,26.75,12.50,12.53,12.53,17.35,18.02,18.78,12.50,12.50,12.53,16.33,16.88,16.20
Training time (s),"7,133.20","6,907.30","7,056.40","6,345.20","6,415.80","6,361.10","8,402.80","8,700.60","8,494.30","7,075.90","6,842.80","6,882.10","8,389.80","8,787.60","8,683.20","7,188.00","6,746.80","6,832.30"
RAM Usage (MB),22.22,44.88,46.71,59.28,45.88,46.35,47.87,40.16,40.00,48.30,48.34,48.58,40.42,19.12,40.52,48.65,18.49,44.76
CPU Usage (s),"7,452.03","7,223.86","7,370.65","6,667.80","6,732.19","6,678.50","8,720.87","9,022.39","8,811.58","7,393.81","7,158.53","7,200.90","8,710.15","9,114.31","9,004.23","7,508.68","7,064.29","7,149.59"
GPU Usage (%),14.98,16.27,15.58,9.43,9.22,8.56,27.10,25.21,25.58,9.71,10.36,10.10,26.63,23.47,26.20,9.32,10.21,10.17
GPU Memory (MB),"3,026.00","3,010.00","3,010.00",310.00,318.00,310.00,"4,348.00","4,348.00","4,348.00",26.00,26.00,26.00,"4,348.00","4,348.00","4,348.00",26.00,26.00,26.00
